<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/weinstein_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stock Screening Inspired by Stan Weinstein

This notebook features stock screening methods inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*. The strategies and criteria used in this analysis are based on the principles outlined in Weinstein's work, aiming to identify potential investment opportunities in both bull and bear markets.


### Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-wlliqoky
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-wlliqoky
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit c9763a890406c5570c3aa6aea70d6f928bc66c64
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.7.0-py3-none-any.whl size=89276 sha256=5a477e577b813c82cb0bce44d73780f33b05ec127096515dc3544c1b66abcf96
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7vnmoml/wheels/37/13/2b/a599ed1c9af840bb5c5d20b1aa8e13e50523bc0d554a13952d
Successfully bu

#### Setup and Configuration

In [2]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [3]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [4]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {dir_path}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='data/weinstein'
)

In [18]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, ma, date = components
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "MA": ma,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [6]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [7]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [8]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [9]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [10]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [11]:
# @title Outputs
outputs = widgets.VBox()

In [12]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `RUI`: Russell 1000
    - `RUT`: Russell 2000
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - `W5000`: Wilshire 5000 Total Market Index
    - `USLS`: U.S. Listed Stocks (美國上市股票)
    - `U.S.Listed`: U.S. Listed Stocks (美國上市股票)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


# Execute Actions Step by Step

## Step 1. Pick a File

In [21]:
# @title File Picker

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown,
        file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='370px'), options=('20241007',), value='20241007')

Dropdown(description='Source:', layout=Layout(width='370px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='Type:', layout=Layout(width='370px'), options=('stocks',), value='stocks')

Dropdown(description='Period:', layout=Layout(width='370px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='370px'), options=(), value=None)

Output()

## Step 2. Filter Stocks

In [22]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond3 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond4 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond5 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS (%)", "Rev RS (%)"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 7
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)

df = github.download_csv(selected_file)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20241007


,Ticker,Name,Sector,Industry,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
4,8937,合騏,Consumer Cyclical,Recreational Vehicles,111.93,124.34,130.26,-15.53,-14.76,-0.32,99.0,67.30,55.24,34.50,0.46,46.93,1.12,46.81,1.649,66.07
13,2543,皇昌,Industrials,Engineering & Construction,94.30,83.94,101.38,45.22,86.21,67.63,98.0,85.80,70.75,54.29,0.98,177.98,4.98,13.74,46.896,18.31
14,4903,聯光通,Technology,Communication Equipment,94.13,56.02,22.20,7.37,-20.11,-8.63,98.0,45.45,30.47,24.54,2.89,944.81,1.15,7.90,7.544,43.43
20,6609,瀧澤科,Technology,Electronic Components,80.04,88.60,72.30,4.47,-11.61,-19.27,98.0,84.50,71.39,51.09,0.44,31.53,2.45,12.60,32.536,35.43
26,1540,喬福,Industrials,Specialty Industrial Machinery,70.85,71.16,72.66,-4.17,-8.49,-7.63,98.0,45.20,42.68,28.97,0.20,29.13,2.09,15.62,7.516,21.63
34,2365,昆盈,Technology,Computer Hardware,58.50,65.83,107.25,82.16,31.46,21.50,97.0,51.30,58.47,41.19,0.10,6.47,0.45,6.22,4.094,117.11
47,6877,鏵友益,Technology,Semiconductor Equipment & Materials,50.77,53.47,80.77,25.89,-25.89,-20.29,97.0,71.60,72.13,50.61,0.14,232.99,0.17,26.66,10.472,434.12
49,2524,京城,Real Estate,Real Estate - Development,49.58,61.43,130.57,16.85,16.42,-9.26,96.0,90.40,112.47,73.48,0.25,38.34,2.56,23.71,10.482,36.21
50,3535,晶彩科,Technology,Computer Hardware,49.27,63.72,112.59,60.96,-14.55,-17.03,96.0,56.40,67.77,45.98,0.15,144.24,1.28,127.36,8.837,44.45
62,4561,健椿,Industrials,Metal Fabrication,42.24,49.97,74.28,39.79,1.91,18.99,96.0,69.50,68.14,53.20,0.10,6.23,4.28,7.99,22.745,16.12


合騏, 皇昌, 聯光通, 瀧澤科, 喬福, 昆盈, 鏵友益, 京城, 晶彩科, 健椿
8937, 2543, 4903, 6609, 1540, 2365, 6877, 2524, 3535, 4561


In [23]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['RS'] > df['1 Week Ago'])" # @param ["& (df['RS'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "& (df['6 Months Ago'] > df['9 Months Ago'])" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond8 = "& (df['EPS RS (%)'] > 0)" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond9 = "& (df['Rev RS (%)'] > 0)" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 12
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20241007


,Ticker,Name,Sector,Industry,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE


In [24]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param [0, 30, 60, 100] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA10'] > 2)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond9 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond10 = "& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))" # @param ["& ((df['EPS RS (%)'] > 0) | (df['Rev RS (%)'] > 0))", ""]
cond11 = "" # @param ["& (df['EPS RS (%)'] > 0)", "& (df['EPS RS (%)'] > 50)", "& (df['EPS RS (%)'] > 100)", "& (df['EPS RS (%)'] > 200)", "& (df['EPS RS (%)'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS (%)'] > 0)", "& (df['Rev RS (%)'] > 5)", "& (df['Rev RS (%)'] > 10)", "& (df['Rev RS (%)'] > 20)", "& (df['Rev RS (%)'] > 50)", "& (df['Rev RS (%)'] > 100)", ""]
cond14 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 15
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)

df = github.download_csv(selected_file)
df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20241007


,Ticker,Name,Sector,Industry,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,MA10,MA30,Volume / VMA10,EPS RS (%),TTM EPS,Rev RS (%),TTM RPS,TTM PE
176,5014,建錩,Basic Materials,Steel,18.48,0.89,-3.16,-5.02,-24.26,-12.62,90.0,19.85,16.56,15.73,6.19,17.42,1.32,0.86,82.334,14.58
319,5288,豐祥-KY,Industrials,Conglomerates,7.55,-13.76,-7.48,-10.88,-7.65,-7.13,82.0,231.50,203.60,202.70,3.71,1.06,16.39,-6.04,109.882,14.28


建錩, 豐祥-KY
5014, 5288


In [27]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 3 # @param [1, 3, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = github.download_csv(selected_file)

# Filter out rows with NaN in the 'Ticker' column
stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS', '1 Week Ago',
              '1 Month Ago', '3 Months Ago', '6 Months Ago', '9 Months Ago']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index()
rating_columns = ['Rating (RS)', 'Rating (1W)',
                  'Rating (1M)', 'Rating (3M)', 'Rating (6M)', 'Rating (9M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: TWSE+TPEX
Type: stocks
Period: 2y
MA: SMA
Date: 20241007


,index,Industry,Sector,Ticker,Name,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Rating (1W),Rating (1M),Rating (3M),Rating (6M),Rating (9M)
0,88,Recreational Vehicles,Consumer Cyclical,"8937,9110,8478","合騏,越南控-DR,東哥遊艇",23.13,26.47,28.99,-18.99,-15.44,-4.22,99,99,99,29,38,60
1,52,Industrial Distribution,Industrials,"8374,911608,2373,3114","羅昇,明輝-DR,震旦行,好德",16.01,15.57,26.24,20.74,-13.97,-10.75,98,97,97,97,45,18
2,60,Internet Content & Information,Communication Services,"3085,5278,5287","新零售,尚凡,數字",15.99,13.63,-2.82,-10.84,-16.21,-13.86,97,96,71,62,35,10


合騏,越南控-DR,東哥遊艇, 羅昇,明輝-DR,震旦行,好德, 新零售,尚凡,數字
8937,9110,8478, 8374,911608,2373,3114, 3085,5278,5287


## Step 3. Visualize Filtered Stocks

In [26]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown()

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [28]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw.stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
